Things to do:

1. build data loader with augmentation
2. CNN architecture
   - transfer learning
3. Train
4. inference

In [2]:
!pip install torchvision



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import time
import copy

import pandas as pd
import numpy as np


from PIL import Image


import torch
from torch import nn, optim

from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Resize,  RandomHorizontalFlip, Normalize, Compose
import torchvision.models as models


/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
!pwd

/root/data/image_classification/notebooks


In [5]:
os.chdir("/root/data/image_classification/")

In [6]:
test_path = "./data/test1"
train_path = "./data/train"
val_path = "./data/val"
training_path = "./data/training"


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
class CatDogDS(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, test_set = None):
        self.animals_df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.test_set = test_set
        if transform:
            self.transform = transform
        else:
            self.transform =  transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
        
    def __len__(self):
        return len(self.animals_df)

    def __getitem__(self, idx):
        
        img_path = os.path.join(self.root_dir,
                                self.animals_df.iloc[idx, 0])

        im = Image.open(img_path) 
        image_ts = self.transform(im)

        
        if self.animals_df.iloc[idx, 1] == 'cat':
            label = 0
        else:
            label = 1
            
        if self.test_set:
            return image_ts, img_path
        else:
        
            return image_ts, label
        


In [9]:
img_transform = transforms.Compose([
        #transforms.RandomSizedCrop(224),
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
    ])




In [10]:
# Build Data loader

In [11]:
train_ds = CatDogDS("./data/training_meta.csv", "./data/training", img_transform, None)
val_ds = CatDogDS("./data/val_meta.csv", "./data/val", None, None)
test_ds = CatDogDS("./data/test_meta.csv", "./data/test1", None, True)

In [12]:
train_dl = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
val_dl = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)
test_dl = DataLoader(test_ds, batch_size=16, shuffle=False)

In [13]:
image_uri = '954362353459.dkr.ecr.us-east-1.amazonaws.com/sm-training-custom:latest'


## Train Model

In [14]:
model = models.vgg16(weights=True)
model.to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [15]:
#next(model.parameters())

In [16]:
def inference (model, val_dl):
    correct_prediction = 0
    total_prediction = 0

    # Disable gradient updates
    with torch.no_grad():
        for data in val_dl:
          # Get the input features and target labels, and put them on the GPU
          inputs, labels = data[0].to(device), data[1].to(device)

          # Normalize the inputs
          inputs_m, inputs_s = inputs.mean(), inputs.std()
          inputs = (inputs - inputs_m) / inputs_s

          # Get predictions
          outputs = model(inputs)

          # Get the predicted class with the highest score
          _, prediction = torch.max(outputs,1)
          # Count of predictions that matched the target label
          correct_prediction += (prediction == labels).sum().item()
          total_prediction += prediction.shape[0]

        acc = correct_prediction/total_prediction
        # print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')
        return acc

In [17]:
def train(model, train_dl, val_dl, num_epochs):
    
    
    criterion = nn.CrossEntropyLoss()
    # Observe that all parameters are being optimized
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    # Decay LR by a factor of 0.1 every 7 epochs
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    val_accuracies = []
    train_accuracies = []

    # Repeat for each epoch
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_prediction = 0
        total_prediction = 0

        # Repeat for each batch in the training set
        for i, data in enumerate(train_dl):
            # Get the input features and target labels, and put them on the GPU
            
            

            inputs, labels = data[0].to(device), data[1].to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Keep stats for Loss and Accuracy
            running_loss += loss.item()

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs,1)

            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

            # if i % 10 == 0:    # print every 10 mini-batches
                # print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))

        # Print stats at the end of the epoch
        num_batches = len(train_dl)
        avg_loss = running_loss / num_batches
        acc = correct_prediction/total_prediction
        
        # Run Model against Validation Set
        val_acc = inference(model, val_dl)
        val_accuracies.append( (epoch, val_acc))
        train_accuracies.append( (epoch, acc))
        
        print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Training Accuracy: {acc:.2f}, Validation Accuracy: {val_acc:.2f}')
        
    return val_accuracies, train_accuracies
        

In [18]:
# Update the last layer to '2' because the output classes are either 0 (cat) or 1 (dot)
# Very important to include .to(device) else this layer gets sent to CPU and will cause an error during training
model.classifier[-1] = nn.Linear(in_features=4096, out_features= 2).to(device) 



In [19]:
val_accuracies, train_accuracies  = train(model, train_dl,val_dl, num_epochs = 3)

Epoch: 0, Loss: 0.17, Training Accuracy: 0.94, Validation Accuracy: 0.96


In [23]:
def make_predictions (model, test_dl):
     # Disable gradient updates
    file_names = []
    predictions = []
    with torch.no_grad():
        for data in test_dl:
            # Get the input features and target labels, and put them on the GPU
            
            inputs, paths = data[0].to(device), data[1]
            
            
            for path in paths:
                file_name = path.split('/')[-1]
                file_names.append(file_name)            
            

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Get predictions
            outputs = model(inputs)

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs,1)
            
            
            for i, x in enumerate(prediction):
                predictions.append(x.item())
                     
            # print(prediction)
    
    return file_names, predictions

In [24]:

file_names, predictions = make_predictions(model, test_dl)

test_predictions_df = pd.DataFrame(list(zip(file_names, predictions)),
           columns =['file_name', 'predicted_class_id'])

In [25]:
# Note the contest submission has been ended. 
test_predictions_df.head()

,file_name,predicted_class_id
0,10592.jpg,0
1,7217.jpg,0
2,3653.jpg,1
3,4382.jpg,0
4,2924.jpg,1


## Research and References

# Competition:
https://www.kaggle.com/competitions/dogs-vs-cats/code

Articles: 
1. https://www.analyticsvidhya.com/blog/2021/06/transfer-learning-using-vgg16-in-pytorch/
2. https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
3. https://www.analyticsvidhya.com/blog/2021/06/transfer-learning-using-vgg16-in-pytorch/

Documentation:
1. https://pytorch.org/vision/stable/generated/torchvision.datasets.ImageFolder.html
2. https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
3. https://pytorch.org/vision/main/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py
4. 